# Import & Set option

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
import re

In [366]:
pd.set_option('display.max_rows', 600)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 2000)
pd.set_option('display.max_colwidth', 1000)

# Read from 64 csv files & Form a Dataframe

In [122]:
filenames=glob('database/*.csv')
data=[]
for f in filenames:
    df=pd.read_csv(f, header=[0,1], delimiter=',')
    data.append(df)

In [123]:
final_df = pd.concat(data, axis=0, ignore_index=True)
final_df.shape

(202525, 97)

In [7]:
#final_df.to_csv('database/data_all.csv')

In [124]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202525 entries, 0 to 202524
Data columns (total 97 columns):
( , ACN)                                                            202525 non-null int64
(Time, Date)                                                        202525 non-null int64
(Time, Local Time Of Day)                                           192620 non-null object
(Place, Locale Reference)                                           198116 non-null object
(Place, State Reference)                                            199960 non-null object
(Place, Relative Position.Angle.Radial)                             20835 non-null float64
(Place, Relative Position.Distance.Nautical Miles)                  97938 non-null float64
(Place, Altitude.AGL.Single Value)                                  81546 non-null object
(Place, Altitude.MSL.Single Value)                                  103514 non-null object
(Environment, Flight Conditions)                                    15506

# Drop duplicates & Drop unnecessary columns

In [125]:
df_nodup=final_df.drop_duplicates()
df_nodup.shape

(202525, 97)

In [320]:
df1=df_nodup.drop(columns=['Work Environment Factor','RVR.Single Value','Aircraft Zone',
                           'Maintenance Status.Maintenance Deferred','Maintenance Status.Records Complete',
                           'Maintenance Status.Released For Service','Maintenance Status.Required / Correct Doc On Board'
                          ,'Maintenance Status.Maintenance Type','Maintenance Status.Maintenance Items Involved',
                          'Cabin Lighting','Crew Size Flight Attendant.Number Of Crew','Callback','When Detected'], level=1)
df1=df_nodup.drop(columns=['Report 2','Unnamed'], level=0)

# Understand the category of Safety Issues from Anomaly column

In [127]:
Anomaly=df1.Events.Anomaly.value_counts(dropna=False)

In [128]:
print(type(Anomaly))
Anomaly.shape

<class 'pandas.core.series.Series'>


(32296,)

In [129]:
idx=pd.Series(range(32296))
Anomaly=pd.DataFrame(Anomaly)
Anomaly.head(50)

,Anomaly
Aircraft Equipment Problem Critical,11266
Other Unspecified,6789
Aircraft Equipment Problem Critical; Other Unspecified,4779
Aircraft Equipment Problem Less Severe,4642
ATC Issue All Types; Conflict Airborne Conflict; Deviation - Procedural Published Material / Policy,2931
Deviation - Procedural Published Material / Policy,2651
Conflict NMAC,2575
ATC Issue All Types; Conflict Airborne Conflict,2414
Deviation - Procedural Clearance; Deviation - Track / Heading All Types,1982
Aircraft Equipment Problem Less Severe; Deviation - Procedural Published Material / Policy,1877


In [130]:
Anomaly['idx']=range(32296)
Anomaly.index.name='Definition'
Anomaly.columns=['Repetition','idx']

# Categorization of Safety Issues in five groups (Conflict,Emergency,Ground,Cabin,Not Categorized)

In [322]:
df2=df1.Events['Anomaly'].fillna('None')

In [323]:
df2

0                        Other No Specific Anomaly Occurred
1         Deviation - Procedural FAR; Other Airspace Vio...
2         Deviation - Procedural FAR; Inflight Event / E...
3                        Other No Specific Anomaly Occurred
4         Aircraft Equipment Problem Less Severe; Deviat...
5                                         Other Unspecified
6           ATC Issue All Types; Conflict Airborne Conflict
7         Deviation - Procedural FAR; Deviation - Track ...
8           ATC Issue All Types; Conflict Airborne Conflict
9         Deviation - Procedural Published Material / Po...
10                                        Other Unspecified
11                       Other No Specific Anomaly Occurred
12                      Aircraft Equipment Problem Critical
13          ATC Issue All Types; Conflict Airborne Conflict
14        Deviation - Altitude Crossing Restriction Not Met
15        Deviation - Altitude Crossing Restriction Not ...
16                   Aircraft Equipment 

In [324]:
assert df2.notnull().all()

In [325]:
df2=pd.DataFrame(df2)

In [326]:
Types=['Ground','Emergency','Conflict','Cabin']
def categorize(dataframe,column_name,new_column='Type'):
    dataframe[new_column]='Not Categorized'
    dataframe.index.fillna('Empty')
    dataframe.fillna('Empty')
    for index, row in dataframe.iterrows():
        if 'Equipment' in dataframe[column_name][index]:
            dataframe[new_column][index]='Emergency'
        elif 'Ground' in dataframe[column_name][index]:
            dataframe[new_column][index]='Ground'
        elif 'Clearence' in dataframe[column_name][index]:
            dataframe[new_column][index]='Conflict'
        elif 'Conflict' in dataframe[column_name][index]:
            dataframe[new_column][index]='Conflict'
        elif 'Deviation' in dataframe[column_name][index]:
            dataframe[new_column][index]='Conflict'
        elif 'Cabin' in dataframe[column_name][index]:
            dataframe[new_column][index]='Cabin'
        else:
            dataframe[new_column][index]='Not Categorized'
    

In [327]:
categorize(df2,'Anomaly','Type')

In [328]:
df2.head()

,Anomaly,Type
0,Other No Specific Anomaly Occurred,Not Categorized
1,Deviation - Procedural FAR; Other Airspace Vio...,Conflict
2,Deviation - Procedural FAR; Inflight Event / E...,Conflict
3,Other No Specific Anomaly Occurred,Not Categorized
4,Aircraft Equipment Problem Less Severe; Deviat...,Emergency


In [329]:
df1=pd.concat([df1,df2],axis=1,join='outer')

In [331]:
df1.head()

,"( , ACN)","(Time, Date)","(Time, Local Time Of Day)","(Place, Locale Reference)","(Place, State Reference)","(Place, Relative Position.Angle.Radial)","(Place, Relative Position.Distance.Nautical Miles)","(Place, Altitude.AGL.Single Value)","(Place, Altitude.MSL.Single Value)","(Environment, Flight Conditions)",...,"(Events, When Detected)","(Events, Result)","(Assessments, Contributing Factors / Situations)","(Assessments, Primary Problem)","(Report 1, Narrative)","(Report 1, Callback)","(Report 1, Synopsis)","(Unnamed: 96_level_0, Unnamed: 96_level_1)",Anomaly,Type
0,79866,198801,1801-2400,CVG,OH,60.0,35.0,NaN,31000,VMC,...,NaN,Air Traffic Control Issued New Clearance,NaN,Human Factors,WE WERE IN OUR CLB FROM CMH TO DFW; WITH A CLR...,NaN,AFTER LEAVING FL320 MLG INSTRUCTED TO DES TO F...,NaN,Other No Specific Anomaly Occurred,Not Categorized
1,79912,198801,1201-1800,SEA; BFI,WA,NaN,6.0,NaN,900,VMC,...,NaN,Flight Crew Exited Penetrated Airspace,NaN,Human Factors,THIS WAS MY FIRST DEP FROM BFI ON 31L. MY TURN...,NaN,SMA PENETRATED TCA ON CLIMB OUT.,NaN,Deviation - Procedural FAR; Other Airspace Vio...,Conflict
2,79965,198801,1201-1800,DEN,CO,NaN,20.0,NaN,7500,VMC,...,NaN,General None Reported / Taken,NaN,Human Factors,A VFR FLT; BEING CONDUCTED UNDER FAR PART 91; ...,NaN,SMT PLT DESCENDED TO ARPT UNDERLYING TCA; ACCU...,NaN,Deviation - Procedural FAR; Inflight Event / E...,Conflict
3,79997,198801,1201-1800,LUK,OH,NaN,5.0,0,NaN,VMC,...,NaN,General None Reported / Taken,NaN,Human Factors,SITUATION BEGAN ON A ROUTINE VFR PLEASURE FLT ...,NaN,ATCT LCL CTLR MIXES UP CALL SIGNS.,NaN,Other No Specific Anomaly Occurred,Not Categorized
4,79998,198801,1201-1800,PUT; BOS,MA,NaN,20.0,NaN,28000,VMC,...,NaN,Air Traffic Control Issued New Clearance; Flig...,NaN,Human Factors,CLRD DIRECT PVT VOR AFTER TKOF BOS. USING R NA...,NaN,FMS STEERED ACFT SOME 20-25 MI OFF COURSE WITH...,NaN,Aircraft Equipment Problem Less Severe; Deviat...,Emergency


In [332]:
Category=df1.Type.value_counts(dropna=False)

In [333]:
Category

Conflict           91669
Emergency          65320
Ground             23084
Not Categorized    20544
Cabin               1908
Name: Type, dtype: int64

Not categorized ones can be categorized by the help of Synopsis column.

In [286]:
#df3=df1['Report 1','Synopsis'].fillna('None')

In [152]:
#df1.to_csv('database/extra/data_categorized.csv')

In [287]:
#df3=df1[df1['Type']=='Not Categorized']
#df3.shape

(202525,)

In [217]:
#df3.info()

In [307]:
#df3=df1['Report 1','Synopsis'].fillna('None')

In [308]:
#print(type(df3))

<class 'pandas.core.series.Series'>


In [309]:
#assert df3.notnull().all()

In [311]:
#df3=pd.DataFrame(df3)
#df3=df3.stack()
#df3=df3.stack()
#df3=pd.DataFrame(df3)
#df3.index.name=['a','b','c']
#df3.columns='d'
#df3

# Only Airspace Control Conflict incidents will be in our scope of interest. Forming the new DF

In [334]:
df_conflict=df1[df1['Type']=='Conflict']

In [353]:
df_conflict.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91669 entries, 1 to 202523
Data columns (total 97 columns):
( , ACN)                                                            91669 non-null int64
(Time, Date)                                                        91669 non-null int64
(Time, Local Time Of Day)                                           88203 non-null object
(Place, Locale Reference)                                           89454 non-null object
(Place, State Reference)                                            90715 non-null object
(Place, Relative Position.Angle.Radial)                             14775 non-null float64
(Place, Relative Position.Distance.Nautical Miles)                  51573 non-null float64
(Place, Altitude.AGL.Single Value)                                  21698 non-null object
(Place, Altitude.MSL.Single Value)                                  63859 non-null object
(Environment, Flight Conditions)                                    74818 non-nu

# Dropping unneccessary columns again :)

In [354]:
df_conflict.index.name='idx'
column_names=list(df_conflict.columns)
df_c1=df_conflict.drop([
  ('Environment', 'Work Environment Factor'),
  ('Environment', 'RVR.Single Value'),
  ('Aircraft 1', 'Aircraft Zone'),
  ('Aircraft 1', 'Maintenance Status.Maintenance Deferred'),
 ('Aircraft 1', 'Maintenance Status.Records Complete'),
 ('Aircraft 1', 'Maintenance Status.Released For Service'),
 ('Aircraft 1', 'Maintenance Status.Required / Correct Doc On Board'),
 ('Aircraft 1', 'Maintenance Status.Maintenance Type'),
 ('Aircraft 1', 'Maintenance Status.Maintenance Items Involved'),
 ('Aircraft 1', 'Cabin Lighting'),
 ('Aircraft 1', 'Crew Size Flight Attendant.Number Of Crew'),
 ('Component', 'Aircraft Component'),
 ('Component', 'Manufacturer'),
 ('Component', 'Aircraft Reference'),
 ('Component', 'Problem'),
 ('Aircraft 2', 'Aircraft Zone'),
 ('Aircraft 2', 'Maintenance Status.Maintenance Deferred'),
 ('Aircraft 2', 'Maintenance Status.Records Complete'),
 ('Aircraft 2', 'Maintenance Status.Released For Service'),
 ('Aircraft 2', 'Maintenance Status.Required / Correct Doc On Board'),
 ('Aircraft 2', 'Maintenance Status.Maintenance Type'),
 ('Aircraft 2', 'Maintenance Status.Maintenance Items Involved'),
 ('Aircraft 2', 'Cabin Lighting'),
 ('Aircraft 2', 'Crew Size Flight Attendant.Number Of Crew'),
 ('Person 1', 'Location In Aircraft'),
 ('Person 1', 'Cabin Activity'),
 ('Person 1', 'Communication Breakdown'),
 ('Person 1', 'ASRS Report Number.Accession Number'),
 ('Person 2', 'Location In Aircraft'),
 ('Person 2', 'Cabin Activity'),
 ('Person 2', 'Human Factors'),
 ('Person 2', 'Communication Breakdown'),
 ('Person 2', 'ASRS Report Number.Accession Number'),
 ('Events', 'Miss Distance'),
 ('Events', 'Were Passengers Involved In Event'),
 ('Events', 'When Detected'),
 ('Report 1', 'Callback'),
 ('Unnamed: 96_level_0', 'Unnamed: 96_level_1'),
 ], axis=1)

In [380]:
column_names=list(df_c1.columns)
del column_names[-1]
del column_names[-1]
column_names

[(' ', 'ACN'),
 ('Time', 'Date'),
 ('Time', 'Local Time Of Day'),
 ('Place', 'Locale Reference'),
 ('Place', 'State Reference'),
 ('Place', 'Relative Position.Angle.Radial'),
 ('Place', 'Relative Position.Distance.Nautical Miles'),
 ('Place', 'Altitude.AGL.Single Value'),
 ('Place', 'Altitude.MSL.Single Value'),
 ('Environment', 'Flight Conditions'),
 ('Environment', 'Weather Elements / Visibility'),
 ('Environment', 'Light'),
 ('Environment', 'Ceiling'),
 ('Aircraft 1', 'ATC / Advisory'),
 ('Aircraft 1', 'Aircraft Operator'),
 ('Aircraft 1', 'Make Model Name'),
 ('Aircraft 1', 'Crew Size'),
 ('Aircraft 1', 'Operating Under FAR Part'),
 ('Aircraft 1', 'Flight Plan'),
 ('Aircraft 1', 'Mission'),
 ('Aircraft 1', 'Nav In Use'),
 ('Aircraft 1', 'Flight Phase'),
 ('Aircraft 1', 'Route In Use'),
 ('Aircraft 1', 'Airspace'),
 ('Aircraft 1', 'Number Of Seats.Number'),
 ('Aircraft 1', 'Passengers On Board.Number'),
 ('Aircraft 2', 'ATC / Advisory'),
 ('Aircraft 2', 'Aircraft Operator'),
 ('Airc

In [367]:
df_c1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91669 entries, 1 to 202523
Data columns (total 59 columns):
( , ACN)                                              91669 non-null int64
(Time, Date)                                          91669 non-null int64
(Time, Local Time Of Day)                             88203 non-null object
(Place, Locale Reference)                             89454 non-null object
(Place, State Reference)                              90715 non-null object
(Place, Relative Position.Angle.Radial)               14775 non-null float64
(Place, Relative Position.Distance.Nautical Miles)    51573 non-null float64
(Place, Altitude.AGL.Single Value)                    21698 non-null object
(Place, Altitude.MSL.Single Value)                    63859 non-null object
(Environment, Flight Conditions)                      74818 non-null object
(Environment, Weather Elements / Visibility)          35445 non-null object
(Environment, Light)                                  

# A new Dataframe with one level columns and Empty cells filled.

In [392]:
df_new={}
for l1,l2 in column_names:
    if df_c1[l1,l2].dtype=='object':
        df_t=df_c1[l1,l2].fillna('None')
        column_t=str(l1)+'_'+str(l2)
        df_new[column_t]=df_t
    if df_c1[l1,l2].dtype=='float64' or df_c1[l1,l2].dtype=='int64':
        df_t=df_c1[l1,l2].fillna(0)
        column_t=str(l1)+'_'+str(l2)
        df_new[column_t]=df_t
len(df_new)

57

In [382]:
data_clean=pd.DataFrame(df_new)

In [387]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91669 entries, 1 to 202523
Data columns (total 57 columns):
_No               91669 non-null int64
Month             91669 non-null int64
Time              91669 non-null object
Place_refer       91669 non-null object
State             91669 non-null object
Radial            91669 non-null float64
Distance          91669 non-null float64
AGL               91669 non-null object
MSL               91669 non-null object
MCondition        91669 non-null object
Visibility        91669 non-null object
Light             91669 non-null object
Ceiling           91669 non-null object
AC1_ATC           91669 non-null object
AC1_Operator      91669 non-null object
AC1_Model         91669 non-null object
AC1_Crew          91669 non-null object
AC1_Rule          91669 non-null object
AC1_FP            91669 non-null object
AC1_Mission       91669 non-null object
AC1_Nav           91669 non-null object
AC1_Phase         91669 non-null object
AC1_Route 

# New Column Names. Much More Convenient :)

In [389]:
column_new_names=['_No','Month','Time','Place_refer','State','Radial','Distance','AGL','MSL','MCondition','Visibility','Light',
                 'Ceiling','AC1_ATC','AC1_Operator','AC1_Model','AC1_Crew','AC1_Rule','AC1_FP','AC1_Mission','AC1_Nav',
                 'AC1_Phase','AC1_Route','AC1_Airspace','AC1_Seats','AC1_Passengers','AC2_ATC','AC2_Operator','AC2_Model',
                  'AC2_Crew','AC2_Rule','AC2_FP','AC2_Mission','AC2_Nav','AC2_Phase','AC2_Route','AC2_Airspace','AC2_Seats',
                  'AC2_Passengers','P1_Loc','P1_Org','P1_Func','P1_Qual','P1_Experience','P1_HumaFactor','P2_Loc','P2_Org',
                  'P2_Func','P2_Qual','P2_Experience','Anomaly','Detector','Result','Other_Factors','Pri_Problem','Narrative',
                  'Synopsis']
data_clean.columns=column_new_names
data_clean.head()

,_No,Month,Time,Place_refer,State,Radial,Distance,AGL,MSL,MCondition,Visibility,Light,Ceiling,AC1_ATC,AC1_Operator,AC1_Model,AC1_Crew,AC1_Rule,AC1_FP,AC1_Mission,AC1_Nav,AC1_Phase,AC1_Route,AC1_Airspace,AC1_Seats,AC1_Passengers,AC2_ATC,AC2_Operator,AC2_Model,AC2_Crew,AC2_Rule,AC2_FP,AC2_Mission,AC2_Nav,AC2_Phase,AC2_Route,AC2_Airspace,AC2_Seats,AC2_Passengers,P1_Loc,P1_Org,P1_Func,P1_Qual,P1_Experience,P1_HumaFactor,P2_Loc,P2_Org,P2_Func,P2_Qual,P2_Experience,Anomaly,Detector,Result,Other_Factors,Pri_Problem,Narrative,Synopsis
idx,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,79912,198801,1201-1800,SEA; BFI,WA,0.0,6.0,None,900,VMC,None,Daylight,None,TRACON SEA; Tower BFI,Personal,Small Aircraft; Low Wing; 1 Eng; Fixed Gear,1,None,VFR,Personal,None,Climb; Takeoff; Initial Climb,None,Class B SEA,0.0,0.0,None,None,None,None,None,None,None,None,None,None,None,0.0,0.0,None,Other Nonprofessional GA,Pilot Flying; Single Pilot,Flight Crew Private,Flight Crew Last 90 Days 5; Flight Crew Total 108; Flight Crew Type 83,None,None,None,Local,Air Traffic Control Fully Certified,None,Deviation - Procedural FAR; Other Airspace Violation Entry or Exit,Person Air Traffic Control,Flight Crew Exited Penetrated Airspace,None,Human Factors,THIS WAS MY FIRST DEP FROM BFI ON 31L. MY TURN TO THE SOUTH WAS PREMATURE. BFI TWR ALERTED ME TO THE PENETRATION AT WHICH TIME A VECTOR WAS GIVEN TO MOVE ME OUT OF THE TCA.,SMA PENETRATED TCA ON CLIMB OUT.
2,79965,198801,1201-1800,DEN,CO,0.0,20.0,None,7500,VMC,20,Daylight,None,TRACON DEN; Tower APA,Personal,Small Transport; Low Wing; 2 Recip Eng,1,None,None,Personal,None,Descent; Descent,None,Class B DEN,0.0,0.0,None,None,None,None,None,None,None,None,None,None,None,0.0,0.0,None,Other Nonprofessional GA,Single Pilot; Pilot Flying,Flight Crew Air Transport Pilot (ATP),Flight Crew Last 90 Days 100; Flight Crew Total 2000; Flight Crew Type 350,None,None,None,None,None,None,Deviation - Procedural FAR; Inflight Event / Encounter Weather / Turbulence; Other Airspace Violation Entry or Exit,Person Air Traffic Control,General None Reported / Taken,None,Human Factors,A VFR FLT; BEING CONDUCTED UNDER FAR PART 91; WAS APCHING DENVER CENTENNIAL ARPT (APA) FROM THE SW. ATIS INDICATED VISUAL APCH TO RWY 35L/R IN USE. DSCNT WAS COMMENCED FROM 17500' MSL; 50 SW OF APA; PLANNING A FINAL APCH ALT OF 7500 MSL WITH A TARGET ALT OF 8000 MSL 20 SW OF DEN. AT APPROX 13000' MSL; 30 SW OF DEN; HDG 060 DEG MAGNETIC; IAS OF 200 KTS; SEVERE TURB WAS ENCOUNTERED (SIGMET FOR SUCH WAS CURRENT). THE ACFT WAS SLOWED TO DESIGN MANEUVERING SPD (155 KTS); TEMPORARILY STOPPING DSCNT. TO AVOID TCA CONFLICT A 30 DEG RIGHT TURN (TO 090 DEG MAGNETIC) WAS MADE. DME INFO FROM DEN VORTAC INDICATED POSITION OUTSIDE TCA. SEVERE TURB AND COCKPIT WORKLOAD PREVENTED RETUNING NAV RECEIVER TO 110.3 MHZ (DEN 26L ILS AND NAV FAC UPON WHICH TCA IS PREDICATED; APPROX 1 1/2 NM SSW OF DEN VORTAC); HOWEVER; VISUAL INFO INDICATED POSITION TO CONTINUE TO BE OUTSIDE OF TCA UNTIL 10000' MSL REACHED WHEN HDG 080 DEG MAGNETIC ASSUMED FOR 'WIDE LEFT BASE; OUTSIDE POWER LINES TO 35R' AS ADVISED BY AP...,SMT PLT DESCENDED TO ARPT UNDERLYING TCA; ACCUSED OF PENETRATING TCA.
6,80007,198801,1201-1800,EMI,MD,0.0,25.0,None,16000,VMC,None,Daylight,None,Center ZDC,Air Carrier,Large Transport; Low Wing; 3 Turbojet Eng,3,None,IFR,Passenger,None,Descent; Descent,None,Class E ZDC,0.0,0.0,None,Air Carrier,Medium Large Transport; Low Wing; 2 Turbojet Eng,2,None,IFR,Passenger,None,Climb; Cruise,None,None,0.0,0.0,None,Government,Enroute,Air Traffic Control Fully Certified,Air Traffic Control Radar 18,None,None,None,Captain; Pilot Flying,Flight Crew Air Transport Pilot (ATP),None,ATC Issue All Types; Conflict Airborne Conflict,Person Air Traffic Control,Air Traffic Control Issued New Clearance; General None Reported / Taken,None,Human Factors,ACR Y CLIMBING TO FL210 WAS STOPPED AT 160 FOR TFC AT 170; WHICH PASSED. ACR Y WAS THEN KEPT AT 160. ACR X WAS DESCENDED TO 170 UNTIL CLEAR O

In [388]:
df=data_clean
#df.to_csv('database/extra/data_clean.csv')